# Sentence Transformer
* 단어가 아닌, 문장 자체를 RoBERTa 기반으로 임베딩하여 유사한 문장을 찾고, 유사한 문서를 클러스터링

In [5]:
from sentence_transformers import SentenceTransformer, models

model_name = "klue/roberta-base"

# 단어 토큰을 임베딩 할 모델 가져오기.
embedding_model = models.Transformer(model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Pooler 정의
모델로 부터 추출된 토큰 단위 임베딩을 가지고 문장 임베딩을 어떻게 계산할 것인지를 결정하는 Pooler를 정의합니다. Pooling 기법은 여러 가지가 있습니다. 여기서는 Mean Pooling을 사용합니다.

Mean Pooling은 모델이 반환한 모든 토큰 임베딩에 대한 평균을 구하여 문장의 임베딩 벡터로 활용하는 기법입니다.

In [6]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),  # 단어 임베딩 모델의 차원을 가져오기
    pooling_mode_mean_tokens=True,  # 풀링을 평균으로 할 수 있게 설정함
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

문장 내 토큰에 대한 임베딩을 수행할 Embedder와 Embedder에 의해 구해진 임베딩 벡터를 문장에 대한 임베딩 벡터로 만들어줄 Pooler를 정의 했으니, SentenceTransformer를 정의할 수 있습니다.

In [7]:
# modules에 정의되는 리스트 내의 원소 순서대로 임베딩 과정이 수행됩니다
#  즉 문장 내 토큰들에 대한 임베딩을 수행한 후 -> pooler를 이용한 문장 임베딩 계산이 일어납니다.
models = SentenceTransformer(modules=[embedding_model, pooler])

# Load Dataset

In [8]:
from datasets import load_dataset

datasets = load_dataset("klue", "sts")
datasets

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 11668
    })
    validation: Dataset({
        features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
        num_rows: 519
    })
})

In [9]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

# 데이터 전처리

* `Datasets` 를 `SentenceTransformer` 양식에 맞게 바꿔주는 작업 수행
* 유사도 점수를 정규화

In [15]:
# InputExample : SentenceTransformer에서 사용되는 데이터 양식으로서 문장과 문장에 대한 결과(label)을 지정할 수 있습니다.

from sentence_transformers.readers import InputExample

train_samples = []
validation_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 까지로 스케일링

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], label=score
        )

        if phase == "validation":
            validation_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [16]:
train_samples[0].texts, train_samples[0].label

(['숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
  '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'],
 0.74)

In [17]:
validation_samples[0].texts, validation_samples[0].label

(['무엇보다도 호스트분들이 너무 친절하셨습니다.', '무엇보다도, 호스트들은 매우 친절했습니다.'], 0.9800000000000001)

# DataLoader 정의

In [28]:
from torch.utils.data import DataLoader

batch_size = 32

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)

validation_dataloader는 따로 만들지 않고, 대신에 모델의 문장 임베딩 간 코사인 유사도가 얼마나 골드 라벨에 가까운지 계산하는 역할을 수행토록 합니다.

즉 train_dataloader를 이용해 훈련한 결과가 validation_dataloader로 Embedding-Pooling 과정을 진행한 후 두 문장의 유사도를 구한 것을 **평가**했을 때 <br>얼마나 잘 따라가는지를 본다고 생각하면 쉽습니다.

In [25]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    validation_samples,
    name="sts-dev",
)

# Training

## warmup step
* train batch data 의 10% 정도를 미리 넣어 모델 학습에 도움
* 준비 운동 단계의 개념  

In [26]:
import math

num_epochs = 4
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

## train

In [31]:
# import torch
from sentence_transformers import losses
from datetime import datetime  # 모델 이름에 날짜 명시

# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# models = models.to(device)

# 문장 간의 cos sim 의 차이를 loss로 정의 (task에 맞는 loss 설정)
train_loss = losses.CosineSimilarityLoss(model=models)

model_save_path = (
    "../../data/saved_models/output/training_klue_sts_"
    + model_name.replace("/", "-")
    + "-"
    + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

models.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

  0%|          | 0/1460 [00:00<?, ?it/s]

KeyboardInterrupt: 

## test set 으로 평가

In [ ]:
# 테스트를 위해 다른 종류의 STS 데이터 세트인 KorSTS 로딩 및 SentenceTransformer에 맞게 처리
testsets = load_dataset("kor_nlu", "sts")

testsets

In [ ]:
test_samples = []

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [ ]:
loaded_model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)

In [ ]:
test_evaluator(loaded_model, output_path=model_save_path)

# Sentence Transformer 활용

## Semantic Search
입력된 문장 간 유사도를 쉽고 빠르게 구할 수 있도록 설계된 `sentence-transformers`를 이용한다면 임베딩을 활용해 다양한 어플리케이션을 고안할 수 있습니다.

먼저 여러 문장 후보군이 주어졌을 때, 입력된 문장과 가장 유사한 문장을 계산하는 예제를 살펴보도록 합시다.

이를 위해 검색의 대상이 되는 문장 후보군을 다음과 같이 정의할 필요가 있습니다. 이후, 정의된 문장 후보군을 미리 임베딩합니다.

In [30]:
docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]

document_embeddings = loaded_model.encode(docs)
document_embeddings

NameError: name 'loaded_model' is not defined

In [ ]:
# test sentence
query = "손흥민은 어린 나이에 유럽에 진출했다."
query_embedding = loaded_model.encode(query)

print(query_embedding)

In [ ]:
from sentence_transformers import util
import torch

top_k = 5

# input sentence - candidate 간 cos sim
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

top_results = torch.topk(cos_scores, k=top_k)
top_results

In [32]:
print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

NameError: name 'query' is not defined

## Clustering

In [35]:
from sklearn.cluster import KMeans

document_embeddings = loaded_model.encode(docs)

num_clusters = 3

k_means = KMeans(n_clusters=num_clusters)
k_means.fit(document_embeddings)

NameError: name 'loaded_model' is not defined

In [37]:
cluster_assignment = k_means.labels_
cluster_assignment

NameError: name 'k_means' is not defined

In [38]:
# 클러스터 개수 만큼 문장을 담을 리스트 초기화
clustered_sentences = [[] for _ in range(num_clusters)]

# 클러스터링 결과를 돌며 각 클러스터에 맞게 문장 삽입
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(docs[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    result = "\n".join(cluster)
    print(f"< 클러스터 {i+1} >\n{result}\n")

NameError: name 'num_clusters' is not defined

In [39]:
# 클러스터 개수 만큼 문장을 담을 리스트 초기화
clustered_sentences = [[] for _ in range(num_clusters)]

# 클러스터링 결과를 돌며 각 클러스터에 맞게 문장 삽입
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(docs[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    result = "\n".join(cluster)
    print(f"< 클러스터 {i+1} >\n{result}\n")

NameError: name 'num_clusters' is not defined

# 참고 - 기존 모델 활용

In [40]:
import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Huffon/sentence-klue-roberta-base")

docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model.encode(docs)

query = "손흥민은 어린 나이에 유럽에 진출하였다."
query_embedding = model.encode(query)

top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

No sentence-transformers model found with name Huffon/sentence-klue-roberta-base. Creating a new one with mean pooling.


config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

입력 문장: 손흥민은 어린 나이에 유럽에 진출하였다.

<입력 문장과 유사한 5 개의 문장>

1: 독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다. (유사도: 0.5897)

2: 그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다. (유사도: 0.4857)

3: 1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다. (유사도: 0.4047)

4: 함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다. (유사도: 0.3953)

5: 춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다. (유사도: 0.3183)

